Start lnd

```
lnd --bitcoin.active --bitcoin.testnet --debuglevel=info --bitcoin.node=neutrino --neutrino.connect=faucet.lightning.community
```

Unlock
```
lncli --network=testnet unlock
```

Gen protobuf code
```
 python -m grpc_tools.protoc --proto_path=googleapis:. --python_out=. --grpc_python_out=. rpc.proto
 ```

In [1]:
import os
import time

import grpc
import codecs

import rpc_pb2 as ln
import rpc_pb2_grpc as lnrpc

In [2]:
# updated ECDSA generated tls.cert
os.environ["GRPC_SSL_CIPHER_SUITES"] = 'HIGH+ECDSA'

In [3]:
lnd_dir = '~/Library/Application Support/Lnd/'
macaroon_file = lnd_dir + "data/chain/bitcoin/testnet/admin.macaroon"
cert_file = lnd_dir + '/tls.cert'

with open(os.path.expanduser(macaroon_file), 'rb') as f:
    macaroon_bytes = f.read()
    macaroon = codecs.encode(macaroon_bytes, 'hex')

with open(os.path.expanduser(cert_file), 'rb') as f:
    cert = f.read()


# create function to automatically add our credentials. For more info see grpc docs.
def metadata_callback(context, callback):
    callback([('macaroon', macaroon)], None)

auth_creds = grpc.metadata_call_credentials(metadata_callback)
cert_creds = grpc.ssl_channel_credentials(cert)

combined_creds = grpc.composite_channel_credentials(cert_creds, auth_creds)

In [4]:
connection = grpc.secure_channel('localhost:10009', combined_creds)
stub = lnrpc.LightningStub(connection)

In [5]:
request = ln.GetInfoRequest()
stub.GetInfo(request)

identity_pubkey: "022e6c6d21aa73495f736e6d1cda53569e5f6f3731f87aaa5c7dc27508819d02e4"
alias: "022e6c6d21aa73495f73"
num_active_channels: 2
num_peers: 3
block_height: 1517341
block_hash: "00000000000000a3eced6bb68e8103d0d19406a7bc8ef99eeeb0cfc84c9fcedc"
synced_to_chain: true
testnet: true
best_header_timestamp: 1558308286
version: "0.6.1-beta commit=v0.6.1-beta-16-gf79a9d97a9a9f0d7c0fdf629d827415fb291dca9"
chains {
  chain: "bitcoin"
  network: "testnet"
}

In [6]:
stub.WalletBalance(ln.WalletBalanceRequest())

total_balance: 6244722
confirmed_balance: 6244722

In [7]:
stub.ListPeers(ln.ListPeersRequest())

peers {
  pub_key: "0270685ca81a8e4d4d01beec5781f4cc924684072ae52c507f8ebe9daf0caaab7b"
  address: "159.203.125.125:9735"
  bytes_sent: 2640
  bytes_recv: 255837
  ping_time: 93998
  sync_type: PASSIVE_SYNC
}
peers {
  pub_key: "023a8dfe081c6bbd0504e599f33d39d17687de63023a8b20afcb59147d9d77c19d"
  address: "92.53.89.123:19735"
  bytes_sent: 59
  bytes_recv: 59
  ping_time: 208369
  sync_type: PASSIVE_SYNC
}
peers {
  pub_key: "02eeb1079cf1f101767504a0e8b9b85fc80b7291625dd880c2d693d960d802120f"
  address: "52.185.134.202:9735"
  bytes_sent: 491
  bytes_recv: 101
  ping_time: 119974
  sync_type: PASSIVE_SYNC
}

In [8]:
request = ln.EstimateFeeRequest(AddrToAmount = {"033bd0c8d5ecdc3685f714d41dfe116e7aedea89e14e4f23c32d33b8a6c03e9c24" : 1_000_000})
stub.EstimateFee(request)

fee_sat: 7700
feerate_sat_per_byte: 50

In [9]:
stub.ListChannels(ln.ListChannelsRequest())

channels {
  active: true
  remote_pubkey: "0270685ca81a8e4d4d01beec5781f4cc924684072ae52c507f8ebe9daf0caaab7b"
  channel_point: "e31fe72b2dd6712955589752f2671c059bbb641f4ef6d8642dc9cce1115f6108:0"
  chan_id: 1666274687536922624
  capacity: 100000
  local_balance: 50000
  remote_balance: 49817
  commit_fee: 183
  commit_weight: 724
  fee_per_kw: 253
  num_updates: 2
  csv_delay: 144
  chan_status_flags: "ChanStatusDefault"
}
channels {
  active: true
  remote_pubkey: "0270685ca81a8e4d4d01beec5781f4cc924684072ae52c507f8ebe9daf0caaab7b"
  channel_point: "ee37ad28e40f4dc979b8f5bd89c9779462108993b58368c131ffb027eab03c4e:0"
  chan_id: 1666279085571244032
  capacity: 500000
  local_balance: 100000
  remote_balance: 399817
  commit_fee: 183
  commit_weight: 724
  fee_per_kw: 253
  num_updates: 2
  csv_delay: 144
  chan_status_flags: "ChanStatusDefault"
}

In [10]:
g = stub.DescribeGraph(ln.ChannelGraphRequest())
for i, n in enumerate(g.nodes[:5]):
    print(f'Node {i}')
    print(n)

Node 0
last_update: 1551676959
pub_key: "02000b1f8a0eac1a299c2594f60055151a1d67bad2c37e09be4fcdc519d7ebe722"
alias: "02000b1f8a0eac1a299c"
color: "#3399ff"

Node 1
pub_key: "02000ddda15408477eb332b535803dc9fa48adba6775f5e11f3ecfd78baa2651b1"
color: "#000000"

Node 2
pub_key: "02006300de37c9177fd137cc06f16d9c351dbdcc6f357912eb7e09f4215d58aa41"
color: "#000000"

Node 3
pub_key: "02006e026ff5dae47924d76a103814d551169a74f562ec8f1760476e063d5bc0ea"
color: "#000000"

Node 4
last_update: 1539877267
pub_key: "0200902b24cb466a068f7e51623d0958f7e47a0daa72e6ad45b8325171ebe129ce"
alias: "0200902b24cb466a068f"
color: "#3399ff"



In [11]:
for i, n in enumerate(g.edges[:5]):
    print(f'Channel {i}')
    print(n)

Channel 0
channel_id: 1313689895797915648
chan_point: "749f88030527b7f729bc8a144c91084b965cfaf9dd8bf7431de0b51965924dda:0"
node1_pub: "022d5a83c275413826b199f847c48316c7c5a89668d102e9103a394b884303e99e"
node2_pub: "02e6f75091d25c9b902b27c77e16e8eba7ebf5e96b71caff2433fbeeb10998469c"
capacity: 16677216

Channel 1
channel_id: 1314245149169352704
chan_point: "8c9424be322f044347fab19327f0fe9847ac8c638c33121ec9199dd4b47d6efd:0"
node1_pub: "02eec2f92298fab9ed3a8f7f9f3519bcff9114a6061122c410a4bb78e279127a8a"
node2_pub: "035b55e3e08538afeef6ff9804e3830293eec1c4a6a9570f1e96a478dad1c86fed"
capacity: 10000000

Channel 2
channel_id: 1314852079589851136
chan_point: "e9ce425cc75e46f7549d7c07f9ef51d405a7f6517c04f188c62aea55871b60bd:0"
node1_pub: "02b4f0aa3fd614644cbe75af230df222d9ec1270ad53ab5123bfec647140bb4708"
node2_pub: "0384c8dc177c5d7c01d8b5d6053699d09ccda823043a572c1a4d1b8949574e57be"
capacity: 16677216

Channel 3
channel_id: 1315069782890774528
chan_point: "5ffdf3820215223ee6b3b730df310a92f13e

In [12]:
request = ln.Invoice(memo="For that thing you owe me", 
                     value=1000000, 
                     expiry=3600,
                     creation_date=int(time.time()))

response = stub.AddInvoice(request)
response

r_hash: "\242z\022\341P\227\207\245\035?\313\013\002p\224\322\235\324\362:\036H\032\024J\004\241\343\211-\030\\"
payment_request: "lntb10m1pwwre7gpp55fap9c2sj7r628flev9syuy562wafu36reyp59z2qjs78zfdrpwqdpggehhygr5dpshggr5dp5kueeq09hh2gr0wajjqmt9cqzpgxqrrss5jy3vla9aelt35vcyyz4xytvqrsvl32umvwadfzwe5zz4crvenz8gr6rjhaa5ca803tl67zz8cnxxts2nugcavu77j6arf3fwykhn7qpf2e22y"
add_index: 8

In [13]:
request = ln.PayReqString(pay_req=response.payment_request)
response = stub.DecodePayReq(request)
response

destination: "022e6c6d21aa73495f736e6d1cda53569e5f6f3731f87aaa5c7dc27508819d02e4"
payment_hash: "a27a12e1509787a51d3fcb0b027094d29dd4f23a1e481a144a04a1e3892d185c"
num_satoshis: 1000000
timestamp: 1558308808
expiry: 3600
description: "For that thing you owe me"
cltv_expiry: 40

In [14]:
stub.ListInvoices(ln.ListInvoiceRequest())

invoices {
  memo: "For that thing you owe me"
  r_preimage: "\027\034\317\221\370vr\216l-\260#\342\353\276\003]K\366\031\354,\273\335?\370\351\324\257O\216\367"
  r_hash: "\263\222\211\001\264\244\275,L\3621\240t\223q\017\343\t\365\271\376O\013\'\362\227I\\\225\005\336~"
  value: 1000000
  creation_date: 1558268333
  payment_request: "lntb10m1pwwzjddpp5kwfgjqd55j7jcn8jxxs8fym3pl3snadele8skfljjay4e9g9melqdpggehhygr5dpshggr5dp5kueeq09hh2gr0wajjqmt9cqzpgxqrrssw67d4wm2dldtutax6twgjm5lx99fdsnu7ckzuv0feluml75pf9angalh93evrdqzz0g8rj7239m8nr8udye5lefndkhwqwqnvm599ugqg7hfxh"
  expiry: 3600
  cltv_expiry: 40
  add_index: 1
}
invoices {
  memo: "For that thing you owe me"
  r_preimage: "\237\372\262Rp\223\022\271\212`mc\251\252\300\261\346C\013Z\301nX\247\257\030\242\026@%|d"
  r_hash: "\214r:\036l&\000E\004w\313\302H\024\341\276|\026c\350\322!u\r\"\023g\217\347g:\214"
  value: 1000000
  creation_date: 1558268355
  payment_request: "lntb10m1pwwzjwrpp533er58nvycqy2prhe0pys98phe7pvclg6gsh2rfzzdncl